In [3]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
%cd ..

c:\Users\haowu\Desktop\Traffic Research\Traffic-Prediction\data_processing


In [2]:
import csv
import pandas as pd
import numpy as np
#import geopandas as gpd
#import networkx as nx
import math
import pickle

import datetime
from datetime import datetime as dt
from collections import Counter
from scipy.spatial.distance import cdist
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from matplotlib import pyplot as plt

from utils import *

### 1. Hyperparameters

In [3]:
# Time
start_date = dt(2022, 2, 10)
end_date = dt(2023, 2, 10)
start_time = 330 # 05:30:00 
end_time = 1260 # 21:00:00
busi_date = pd.bdate_range(start=start_date, end=end_date).date 

In [5]:
'''
execute the following code block only if set_hwd_tmc_segments_shp and set_hwd_xd_segments_str_shp are not in place
'''
# # Segments
# gdf_hwd_tmc = gpd.read_file("/Volumes/Hao's WD/Howard/Shapefiles/tmc_shape_hwd.geojson")  # 1093 TMC segments in Howard, MD 
# gdf_hwd_xd = gpd.read_file("/Volumes/Hao's WD/Howard/Shapefiles/xd_shape_hwd_for_sjoin.geojson")  # 1772 of 4723 XD segments in Howard, MD; used for spatial join

# set_hwd_tmc_segments_shp = set(gdf_hwd_tmc.tmc)
# set_hwd_xd_segments_str_shp = set(gdf_hwd_xd.xd.astype(str))

# pickle.dump(set_hwd_tmc_segments_shp, open("./Maryland/set_hwd_tmc_segments_shp.pkl", "wb"))
# pickle.dump(set_hwd_xd_segments_str_shp, open("./Maryland/set_hwd_xd_segments_str_shp.pkl", "wb"))

'\nexecute the following code block only if set_hwd_tmc_segments_shp and set_hwd_xd_segments_str_shp are not in place\n'

In [4]:
df_hwd_xd_to_tmc = pickle.load(open("./Maryland/df_hwd_xd_to_tmc.pkl", "rb"))
set_hwd_xd_segments_sjoin = set(df_hwd_xd_to_tmc.id_xd_str)  # 1711

### 2. Downsample Raw Speed Data Files

In [8]:
'''
Execute this code block ONLY IF downsampled csv data are not available on file
tmc process and xd process cannot be executed simultaneously, otherwise the notebook will encounter kernel crash
'''

# # Take ~23 min to execute (~9 min in reading chunks (58 iterations), ~13 min in writing into dowmsampled csv file)
# # downsampled_tmc_1min_all: shape (265241843, 8), contains all 1093 TMC segments as in set_hwd_tmc_segments_shp
# downsampled_tmc_1min_all = downsample_large_dataset("tmc", "/Volumes/Hao's WD/Howard/Speed/howard_tmc_1min/hwd_tmc_1min_all.csv", "/Volumes/Hao's WD/Howard/Speed/howard_tmc_1min/downsampled_hwd_tmc_1min_all.csv", set_hwd_tmc_segments_shp, busi_date, start_time, end_time)

# # Take ~64 min to execute (~45 min in reading chunks (248 iterations), ~ 19 min in writing into dowmsampled csv file)
# # downsampled_xd_1min: shape (431282061, 8), contains all 1772 XD segments as in set_hwd_xd_segments_str_shp
# downsampled_xd_1min = downsample_large_dataset("xd", "/Volumes/Hao's WD/Howard/Speed/howard_xd_1min/hwd_xd_1min.csv", "/Volumes/Hao's WD/Howard/Speed/howard_xd_1min/downsampled_hwd_xd_1min.csv", set_hwd_xd_segments_str_shp, busi_date, start_time, end_time)

# # Take ~27 min to execute (~9 min in reading chunks, ~18 min in writing into dowmsampled csv file)
# downsampled_xd_1min = downsample_large_dataset("xd", "../data/downsampled_hwd_xd_1min.csv", "../data/downsampled_hwd_xd_1min_1711.csv", set_hwd_xd_segments_sjoin, busi_date, start_time, end_time)

OSError: [Errno 5] Input/output error: "/Volumes/Hao's WD/Howard/Speed/howard_xd_1min/downsampled_hwd_xd_1min.csv"

In [5]:
# ~26 GB 
downsampled_xd_1min_1711 = pd.read_csv("../data/downsampled_hwd_xd_1min_1711.csv")

In [10]:
# ~16 GB
downsampled_tmc_1min_all = pd.read_csv("../data/downsampled_hwd_tmc_1min_all.csv")

### 3. Pivot Downsampled Data and Generate Speed/Density Dataframe
The following code blocks are executed on MSI desktop, which has 64GB RAM. 

In [7]:
df_downsampled_spd_xd_1min = pivot_df("xd", "speed", 1, df=downsampled_xd_1min_1711, busi_date=busi_date, num_slot=930, freq=1, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_hwd_spd_xd_1min_1711.pkl") 

Check dates of incomplete slots:
2022-03-24 926
2022-04-14 922
2022-04-27 928
2022-04-29 926
2022-05-02 927
2022-05-04 924
2022-05-05 929
2022-06-16 929
2022-06-22 928
2022-07-07 929
2022-07-08 929
2022-08-02 885
2022-08-03 883
2022-08-10 929
2022-08-17 929
2022-10-12 926
2022-10-19 929
2022-11-17 866
2022-11-21 929
2022-12-01 928
2022-12-14 917
2022-12-15 928
2022-12-16 926
2022-12-22 927
2022-12-23 926
2022-12-30 926
2023-01-03 929
2023-01-19 928
2023-02-03 929


In [8]:
df_downsampled_dens_xd_1min = pivot_df("xd", "density", 1, df=downsampled_xd_1min_1711, busi_date=busi_date, num_slot=930, freq=1, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_hwd_dens_xd_1min_1711.pkl") 

Check dates of incomplete slots:
2022-03-24 926
2022-04-14 922
2022-04-27 928
2022-04-29 926
2022-05-02 927
2022-05-04 924
2022-05-05 929
2022-06-16 929
2022-06-22 928
2022-07-07 929
2022-07-08 929
2022-08-02 885
2022-08-03 883
2022-08-10 929
2022-08-17 929
2022-10-12 926
2022-10-19 929
2022-11-17 866
2022-11-21 929
2022-12-01 928
2022-12-14 917
2022-12-15 928
2022-12-16 926
2022-12-22 927
2022-12-23 926
2022-12-30 926
2023-01-03 929
2023-01-19 928
2023-02-03 929


In [11]:
df_downsampled_spd_tmc_1min_all = pivot_df("tmc", "speed", 1, df=downsampled_tmc_1min_all, busi_date=busi_date, num_slot=930, freq=1, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_hwd_spd_tmc_1min_all.pkl") 

Check dates of incomplete slots:
2022-02-10 924
2022-03-24 926
2022-04-14 924
2022-04-21 927
2022-04-22 928
2022-04-25 929
2022-04-28 929
2022-04-29 926
2022-05-02 927
2022-05-03 929
2022-05-04 925
2022-05-06 925
2022-05-09 928
2022-05-10 929
2022-05-11 927
2022-05-12 928
2022-05-16 929
2022-05-18 928
2022-05-20 929
2022-05-23 929
2022-05-24 928
2022-05-25 929
2022-05-26 929
2022-05-27 928
2022-05-30 928
2022-05-31 929
2022-06-01 928
2022-06-02 928
2022-06-03 929
2022-06-06 926
2022-06-15 929
2022-06-16 929
2022-06-17 927
2022-06-22 927
2022-06-23 929
2022-06-24 928
2022-06-28 929
2022-07-01 929
2022-07-29 929
2022-08-02 894
2022-08-03 892
2022-08-16 929
2022-08-17 929
2022-10-03 929
2022-10-12 927
2022-10-13 928
2022-10-14 928
2022-10-17 928
2022-10-19 929
2022-11-17 877
2022-11-21 929
2022-12-01 926
2022-12-07 929
2022-12-09 928
2022-12-15 926
2022-12-16 925
2022-12-21 928
2022-12-22 926
2022-12-23 927
2022-12-27 929
2022-12-30 927
2023-01-03 929
2023-01-18 929
2023-01-26 929
2023-02

In [12]:
df_downsampled_dens_tmc_1min_all = pivot_df("tmc", "density", 1, df=downsampled_tmc_1min_all, busi_date=busi_date, num_slot=930, freq=1, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_hwd_dens_tmc_1min_all.pkl") 

Check dates of incomplete slots:
2022-02-10 924
2022-03-24 926
2022-04-14 924
2022-04-21 927
2022-04-22 928
2022-04-25 929
2022-04-28 929
2022-04-29 926
2022-05-02 927
2022-05-03 929
2022-05-04 925
2022-05-06 925
2022-05-09 928
2022-05-10 929
2022-05-11 927
2022-05-12 928
2022-05-16 929
2022-05-18 928
2022-05-20 929
2022-05-23 929
2022-05-24 928
2022-05-25 929
2022-05-26 929
2022-05-27 928
2022-05-30 928
2022-05-31 929
2022-06-01 928
2022-06-02 928
2022-06-03 929
2022-06-06 926
2022-06-15 929
2022-06-16 929
2022-06-17 927
2022-06-22 927
2022-06-23 929
2022-06-24 928
2022-06-28 929
2022-07-01 929
2022-07-29 929
2022-08-02 894
2022-08-03 892
2022-08-16 929
2022-08-17 929
2022-10-03 929
2022-10-12 927
2022-10-13 928
2022-10-14 928
2022-10-17 928
2022-10-19 929
2022-11-17 877
2022-11-21 929
2022-12-01 926
2022-12-07 929
2022-12-09 928
2022-12-15 926
2022-12-16 925
2022-12-21 928
2022-12-22 926
2022-12-23 927
2022-12-27 929
2022-12-30 927
2023-01-03 929
2023-01-18 929
2023-01-26 929
2023-02